In [2]:
import pandas as pd
import os

In [3]:
output_dir = '/workspaces/airoute_mlops/airoute_mlops/data/metadata'
df = os.listdir('/workspaces/airoute_mlops/airoute_mlops/data/bronze/defra')
df = pd.DataFrame(df, columns=['file_name'])
df['file_name'] = df['file_name'].str.replace('.csv', '')
output_path = os.path.join(output_dir, 'active_sites.csv')
print(df)
df.to_csv(output_path, index=False)
print("Active sites saved to active_sites.csv")

     file_name
0    BLWD_2025
1    WAL4_2025
2    PEGR_2025
3    SOUT_2025
4     CAM_2025
..         ...
182  BR11_2025
183  EDNS_2025
184  LVPT_2025
185  SEND_2025
186  LONM_2025

[187 rows x 1 columns]
Active sites saved to active_sites.csv


In [4]:
meta_sites = pd.read_csv(r'/workspaces/airoute_mlops/airoute_mlops/data/metadata/site_list.csv')
meta_sites.head()

,site_id,site_name,location_type,latitude,longitude,no2,o3,pm25
0,A3,London A3 Roadside,Urban Traffic,51.373480,-0.291853,1.0,0.0,0.0
1,ABD,Aberdeen,Urban Background,57.157360,-2.094278,1.0,1.0,1.0
2,ABD7,Aberdeen Union Street Roadside,Urban Traffic,57.144555,-2.106472,1.0,0.0,0.0
3,ABD8,Aberdeen Wellington Road,Urban Traffic,57.133888,-2.094198,1.0,0.0,0.0
4,ABD9,Aberdeen Erroll Park,Urban Background,57.157400,-2.094770,1.0,1.0,1.0


In [5]:
df

,file_name
0,BLWD_2025
1,WAL4_2025
2,PEGR_2025
3,SOUT_2025
4,CAM_2025
...,...
182,BR11_2025
183,EDNS_2025
184,LVPT_2025
185,SEND_2025


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  187 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


In [7]:
act_site_df = df['file_name'].str.strip('_2025') 
act_site_df = pd.DataFrame(act_site_df, columns=['file_name'])
act_site_df['file_name'] = act_site_df.drop_duplicates()
print(act_site_df)


    file_name
0        BLWD
1        WAL4
2        PEGR
3        SOUT
4         CAM
..        ...
182      BR11
183      EDNS
184      LVPT
185      SEND
186      LONM

[187 rows x 1 columns]


In [8]:
# act_site_df('HULL').isin(act_site_df['file_name'])
# act_site_df[act_site_df['file_name']
'INV' in act_site_df['file_name'].values

True

In [9]:
act_site_df.head()

,file_name
0,BLWD
1,WAL4
2,PEGR
3,SOUT
4,CAM


In [10]:
meta_sites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   site_id        270 non-null    object 
 1   site_name      270 non-null    object 
 2   location_type  270 non-null    object 
 3   latitude       270 non-null    float64
 4   longitude      270 non-null    float64
 5   no2            270 non-null    float64
 6   o3             270 non-null    float64
 7   pm25           270 non-null    float64
dtypes: float64(5), object(3)
memory usage: 17.0+ KB


In [11]:
meta_sites.head()

,site_id,site_name,location_type,latitude,longitude,no2,o3,pm25
0,A3,London A3 Roadside,Urban Traffic,51.373480,-0.291853,1.0,0.0,0.0
1,ABD,Aberdeen,Urban Background,57.157360,-2.094278,1.0,1.0,1.0
2,ABD7,Aberdeen Union Street Roadside,Urban Traffic,57.144555,-2.106472,1.0,0.0,0.0
3,ABD8,Aberdeen Wellington Road,Urban Traffic,57.133888,-2.094198,1.0,0.0,0.0
4,ABD9,Aberdeen Erroll Park,Urban Background,57.157400,-2.094770,1.0,1.0,1.0


In [12]:
# Merge act_site_df and meta_sites on site_id/file_name
active_locations = pd.merge(
    act_site_df,
    meta_sites,
    left_on='file_name',
    right_on='site_id',
    how='inner'
)
print(active_locations)

    file_name site_id                    site_name     location_type  \
0        BLWD    BLWD    Blackpool Whitegate Drive     Urban Traffic   
1        WAL4    WAL4            Walsall Woodlands  Urban Background   
2        PEGR    PEGR           Perth Glasgow Road     Urban Traffic   
3        SOUT    SOUT           Southampton Centre  Urban Background   
4         CAM     CAM           Cambridge Roadside     Urban Traffic   
..        ...     ...                          ...               ...   
171      BR11    BR11           Bristol Temple Way     Urban Traffic   
172      EDNS    EDNS    Edinburgh Nicolson Street     Urban Traffic   
173      LVPT    LVPT     Liverpool Thirlmere Park  Urban Background   
174      SEND    SEND              Southend-on-Sea  Urban Background   
175      LONM    LONM  London Norbury Manor School  Urban Background   

      latitude  longitude  no2   o3  pm25  
0    53.810975  -3.033232  0.0  0.0   1.0  
1    52.605641  -2.030371  1.0  1.0   0.0  
2  

In [13]:
not_merged = act_site_df.merge(
    meta_sites,
    left_on='file_name',
    right_on='site_id',
    how='left',
    indicator=True
)
not_merged = not_merged[not_merged['_merge'] == 'left_only']
print(not_merged)

    file_name site_id site_name location_type  latitude  longitude  no2  o3  \
8         WIG     NaN       NaN           NaN       NaN        NaN  NaN NaN   
14        BEL     NaN       NaN           NaN       NaN        NaN  NaN NaN   
38        YK1     NaN       NaN           NaN       NaN        NaN  NaN NaN   
66         SK     NaN       NaN           NaN       NaN        NaN  NaN NaN   
75        HUL     NaN       NaN           NaN       NaN        NaN  NaN NaN   
84        BLC     NaN       NaN           NaN       NaN        NaN  NaN NaN   
103       NO1     NaN       NaN           NaN       NaN        NaN  NaN NaN   
131        TH     NaN       NaN           NaN       NaN        NaN  NaN NaN   
136       REA     NaN       NaN           NaN       NaN        NaN  NaN NaN   
149       INV     NaN       NaN           NaN       NaN        NaN  NaN NaN   
173       SUN     NaN       NaN           NaN       NaN        NaN  NaN NaN   

     pm25     _merge  
8     NaN  left_only  
14   

In [14]:
active_locations.head()

,file_name,site_id,site_name,location_type,latitude,longitude,no2,o3,pm25
0,BLWD,BLWD,Blackpool Whitegate Drive,Urban Traffic,53.810975,-3.033232,0.0,0.0,1.0
1,WAL4,WAL4,Walsall Woodlands,Urban Background,52.605641,-2.030371,1.0,1.0,0.0
2,PEGR,PEGR,Perth Glasgow Road,Urban Traffic,56.388236,-3.478055,1.0,0.0,0.0
3,SOUT,SOUT,Southampton Centre,Urban Background,50.908173,-1.395757,1.0,1.0,1.0
4,CAM,CAM,Cambridge Roadside,Urban Traffic,52.202370,0.124456,1.0,0.0,0.0


In [17]:
# Select the desired columns
cols_to_save = ['site_id', 'site_name', 'location_type', 'latitude', 'longitude']
active_locations_selected = active_locations[cols_to_save]

# Save to csv
csv_parquet = '/workspaces/airoute_mlops/airoute_mlops/data/metadata/active_sites1.csv'
active_locations_selected.to_csv(csv_parquet, index=False)